# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [32]:
import os
import shutil
import joblib

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.automl.core.shared import constants

## Dataset

### Overview

In this problem, we are using [Heart Failure Prediction](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) dataset.
from Kaggle.
The dataset has the below set of 12 features and a target variable :

- age: Age of the patient
- amaemia: Decrease of red blood cells or hemoglobin
- creatinine_phosphokinase: Level of the CPK enzyme in the blood (mcg/L)
- diabetes: If the patient has diabetes
- ejection_fraction: Percentage of blood leaving the heart at each contraction
- high_blood_pressure: If the patient has hypertension
- platelets: Platelets in the blood (kiloplatelets/mL)
- serum_creatinine: Level of serum creatinine in the blood (mg/dL)
- serum_sodium: Level of serum sodium in the blood (mEq/L)
- sex: Woman or man
- smoking: If the patient smokes or not
- time: Follow-up period (days)
- DEATH_EVENT - If the patient deceased during the follow-up period (boolean)

In [33]:
ws = Workspace.from_config()

experiment_name = 'heart-failure-prediction-hyperdrive'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [34]:
print('Workspace name: ' + ws.name, 
      'Subscription id: ' + ws.subscription_id, sep = '\n')

Workspace name: udacity-azure-ml
Subscription id: 7292cf53-4607-42aa-ae5d-bbb29bb79c07


In [35]:
# Create compute cluster
compute_cluster_name= "automl-compute"

# Check if compute cluster already exists, else create new Compute Cluster
try:
    compute_cluster=ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, using it")
except ComputeTargetException:
    print("Creating new cluster")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=5)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [36]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
    evaluation_interval=1,
    slack_factor= 0.1
)

# Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        "--C": uniform(0.001, 100),
        "--max_iter": choice(50, 75, 100, 125, 150, 175, 200)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")
    
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)

# Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory= script_folder,
    compute_target= compute_cluster,
    entry_script= "train.py",
    vm_size="Standard_D2_V2",
    vm_priority="lowpriority"
)

hyperdrive_run_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy= early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=5
)

In [37]:
# Submit your experiment
hyperdrive_run=experiment.submit(config=hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [42]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [43]:
hyperdrive_run.wait_for_completion(show_output= True)

RunId: HD_762932d8-dc58-4c9f-a6b0-814edce933dd
Web View: https://ml.azure.com/runs/HD_762932d8-dc58-4c9f-a6b0-814edce933dd?wsid=/subscriptions/7292cf53-4607-42aa-ae5d-bbb29bb79c07/resourcegroups/udacity-azure-ml/workspaces/udacity-azure-ml&tid=6f4ead32-2680-469c-991a-2b4d707c6cc9

Execution Summary
RunId: HD_762932d8-dc58-4c9f-a6b0-814edce933dd
Web View: https://ml.azure.com/runs/HD_762932d8-dc58-4c9f-a6b0-814edce933dd?wsid=/subscriptions/7292cf53-4607-42aa-ae5d-bbb29bb79c07/resourcegroups/udacity-azure-ml/workspaces/udacity-azure-ml&tid=6f4ead32-2680-469c-991a-2b4d707c6cc9



{'runId': 'HD_762932d8-dc58-4c9f-a6b0-814edce933dd',
 'target': 'automl-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-09-03T11:23:09.231208Z',
 'endTimeUtc': '2021-09-03T11:30:43.590511Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'a6b6d311-ba36-4e0a-8de8-99bbad7552d3',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.9',
  'best_child_run_id': 'HD_762932d8-dc58-4c9f-a6b0-814edce933dd_6',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://udacityazureml9980995583.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_762932d8-dc58-4c9f-a6b0-814edce933dd/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=b&s

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [50]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(best_run.get_details()['runDefinition']['arguments'])
print('Best Run Accuracy:', best_run_metrics['Accuracy'])
print('Best Run ID:', best_run.id)

['--C', '58.69682498001539', '--max_iter', '200']
Best Run Accuracy: 0.9
Best Run ID: HD_762932d8-dc58-4c9f-a6b0-814edce933dd_6


In [45]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_ae554c31bfbd80e5172f204fcd8f0fb5262e135f9bf4bd0c2dec57cccb88ba72_d.txt',
 'azureml-logs/65_job_prep-tvmps_ae554c31bfbd80e5172f204fcd8f0fb5262e135f9bf4bd0c2dec57cccb88ba72_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_ae554c31bfbd80e5172f204fcd8f0fb5262e135f9bf4bd0c2dec57cccb88ba72_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/110_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/hyperDrive_58.69682498001539_200']

In [46]:
# Save and register the best model
model = best_run.register_model(model_name='heart-failure-sklearn-hypertuned', model_path='outputs/hyperDrive_58.69682498001539_200')
best_run.download_file('/outputs/hyperDrive_58.69682498001539_200', 'hyperdrive_model_best.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service